In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime
from datetime import date
import csv
import matplotlib.pyplot as plt
import math
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
from IPython.display import display_html
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [102]:
#Loading MAPPING-related tables

In [3]:
device_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLALLDEVICEDATA_DPL.csv', sep = '|', encoding = "ISO-8859-1")
    #Gives:  Detailed descriptions/features of each device
#EXCLUDED:  city_df = pd.read_csv('gs://aes-datahub-0001-landing/OMS/DPL/TCMS.TBLCSSCITY.csv', sep = ';')
download_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TCMS.TBLCSSDOWNLOAD_A_DPL.csv', sep = '|')
    #Gives:  'Account Type'
    #Note:  There are 598352 unique KY_PREM's
#EXCLUDED:  prem_df = pd.read_csv('gs://aes-datahub-0001-landing/OMS/DPL/TCMS.TBLCSSPREMDATA.csv', sep = ';')
callsource_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLLOOKUPCALLSOURCE_DPL.csv', sep = '|')
calltype_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLLOOKUPCALLTYPES_DPL.csv', sep = '|')
causecode_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLLOOKUPCAUSECODE.csv', sep = '|')
    #Gives:  CAUSE_CODE, CAUSEDESCRIPTION
faileddevice_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLLOOKUPFAILEDDEVICE_DPL.csv', sep = '|')
    #Gives:  FAILED_DEVICE_CODE, FAILED_DEVICE
interruptingdevice_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLLOOKUPINTERRUPTINGDEVICE_DPL.csv', sep = '|')
    #Gives:  INT_CODE, INTERRUPTING_DEVICE
#EXCLUDED:  servicecenter_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/DPL/2002-2020/TBLLOOKUPSERVICECENTER_DPL.csv', sep = '|')
    #Gives:  The service center and the hierarchy that it belongs to

In [4]:
device_df.head()

,ID,CIRCUIT,DEVICE,UPLINE,PHASE_A,PHASE_B,PHASE_C,PHASE_T,DEVICETYPE,DEVICESIZE,GRID,GRID_COORD,LOCATION,REMARK,TOPFUSE,LENGTH,UNDERGD,SERVCTR,NORMALLYOPEN,CIRCUITTIE,SORTNUMBER,SORTLEVEL,TIEUPLINE,VOLTAGE,DIRECTIONFWD,LATITUDE,LONGITUDE,MAP_GRID,MAP_GRID_COORD,MAP_FILE,MAP_SIZE,NSSORTNUMBER,NSSORTLEVEL,NSUPLINE,NSDIRECTSPLITS_A,NSDIRECTSPLITS_B,NSDIRECTSPLITS_C,NSDIRECTSPLITS_T,CUST_A,CUST_B,CUST_C,CUST_T,TOTCUST_A,TOTCUST_B,TOTCUST_C,TOTCUST_T,KVA_A,KVA_B,KVA_C,TOTKVA_A,TOTKVA_B,TOTKVA_C,TOTMAJ_A,TOTMAJ_B,TOTMAJ_C,TOTMAJ_T,TOTMED_A,TOTMED_B,TOTMED_C,TOTMED_T,TOTUTL_A,TOTUTL_B,TOTUTL_C,TOTUTL_T,TOTEMR_A,TOTEMR_B,TOTEMR_C,TOTEMR_T,TOTGOV_A,TOTGOV_B,TOTGOV_C,TOTGOV_T,TOTLRT_A,TOTLRT_B,TOTLRT_C,TOTLRT_T,TOTLFS_A,TOTLFS_B,TOTLFS_C,TOTLFS_T,IVRSORTNUMBER,LASTUPDATE,DATETIMEADDED,TOTNONRES_A,TOTNONRES_B,TOTNONRES_C,TOTNONRES_T,INDEX_FLAG
0,80480,MC1201,E8998,C2693,-1.0,NaN,NaN,NaN,F RISER,20K,B32/23,C-1,514 Duke Rd,NaN,NaN,NaN,NaN,5.0,NaN,NaN,MC1201010301,3.0,NaN,12KV (100%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,MC12010101,2.0,MC-01,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1,0,0,0,100.0,NaN,NaN,100.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,020,09-07-2020 03:18:54,21-12-2018 09:45:41,0.0,0.0,0.0,0.0,NaN
1,11220,AN1202,AN-02,NaN,-1.0,-1.0,-1.0,-1.0,CKT BKR,1200A,E33/42,E-4,HUBER HEIGHTS SUBSTATION BREAKER,NaN,-1.0,0.0,0.0,9.0,0.0,NaN,AN120201,1.0,NaN,12KV (100%),0.0,39.842465,-84.101442,E33/42,E-4,NaN,NaN,AN120201,1.0,NaN,1.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,523,619,375,143,50.0,0.0,50.0,7982.0,9256.0,6189.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,04-06-2020 03:18:22,NaN,0.0,0.0,0.0,0.0,NaN
2,11221,AN1201,AN-01,NaN,-1.0,-1.0,-1.0,-1.0,CKT BKR,1200A,E33/42,E-4,HUBER HEIGHTS SUBSTATION,NaN,-1.0,0.0,0.0,9.0,0.0,NaN,AN120101,1.0,NaN,12KV (100%),0.0,39.842465,-84.101442,E33/42,E-4,NaN,NaN,AN120101,1.0,NaN,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,92,83,141,9,0.0,0.0,0.0,4788.0,4165.0,4750.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,05-06-2020 03:17:44,NaN,7.0,15.0,10.0,12.0,NaN
3,11222,AN1204,AN-04,NaN,-1.0,-1.0,-1.0,-1.0,CKT BKR,1200A,E33/42,E4,HUBER HEIGHTS SUBSTATION BREAKER,NaN,-1.0,0.0,0.0,9.0,0.0,NaN,AN120401,1.0,NaN,12KV (100%),0.0,39.842465,-84.101442,E33/42,E-4,NaN,NaN,AN120401,1.0,NaN,1.0,1.0,1.0,1.0,1.0,0.0,0.0,39.0,752,556,921,147,0.0,0.0,0.0,7040.0,6757.0,10157.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,4,3,0,0,07-07-2020 03:20:22,NaN,58.0,50.0,64.0,94.0,NaN
4,11224,AN1206,AN-06,NaN,-1.0,-1.0,-1.0,-1.0,CKT BKR,1200A,E33/42,E-4,HUBER HEIGHTS SUBSTATION BREAKER,NaN,-1.0,0.0,0.0,9.0,0.0,NaN,AN120601,1.0,NaN,12KV (100%),0.0,39.842465,-84.101442,E33/42,E-4,NaN,NaN,AN120601,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,829,390,467,54,0.0,0.0,0.0,6994.0,5127.0,5582.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,03-07-2020 03:17:14,NaN,0.0,0.0,0.0,0.0,NaN


In [5]:
#Selecting MAPPING variables

In [6]:
device_df = device_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86]]

In [7]:
print("QC check 1", device_df.shape)
print("QC check 1", device_df.iloc[:, 0].nunique())

QC check 1 (36858, 67)
QC check 1 36858


In [8]:
download_df = download_df[['KY_PREM', 
                           'ACCOUNT_TYPE']]

In [9]:
print("QC check 2", download_df.shape)
print("QC check 2", download_df.iloc[:, 0].nunique())

QC check 2 (600123, 2)
QC check 2 600123


In [10]:
callsource_df = callsource_df[['CALLSOURCEID',
                               'SOURCE_DESCRIPTION']]

In [11]:
print("QC check 3", callsource_df.shape)
print("QC check 3", callsource_df.iloc[:, 0].nunique())

QC check 3 (5, 2)
QC check 3 5


In [12]:
#Likely can be excluded
calltype_df = calltype_df[calltype_df['HIDE'] == 0][['CALL_TYPE', 
                                                     'ID']]

In [13]:
print("QC check 4", calltype_df.shape)
print("QC check 4", calltype_df.iloc[:, 0].nunique())

QC check 4 (11, 2)
QC check 4 11


In [14]:
causecode_df = causecode_df[causecode_df['HIDE'] == 0][['CAUSEID', 
                                                        'CAUSE_CODE',
                                                        'CAUSEDESCRIPTION',
                                                        'NOTRANSOPS',
                                                        'NODISTRIBUTION',
                                                        'OUTAGE_FOLLOWUP_CALL']]

In [15]:
print("QC check 5", causecode_df.shape)
print("QC check 5", causecode_df.iloc[:, 0].nunique())

QC check 5 (40, 6)
QC check 5 40


In [16]:
faileddevice_df = faileddevice_df[faileddevice_df['HIDE'] == 0][['ID', 
                                                                 'FAILED_DEVICE_CODE', 
                                                                 'FAILED_DEVICE']]

In [17]:
print("QC check 6", faileddevice_df.shape)
print("QC check 6", faileddevice_df.iloc[:, 0].nunique())

QC check 6 (43, 3)
QC check 6 43


In [18]:
faileddevice_df = faileddevice_df.rename(columns = {'ID': 'FAILED_DEVICE_ID'})

In [19]:
interruptingdevice_df = interruptingdevice_df[interruptingdevice_df['HIDE'] == 0][['ID',
                                                                                   'INT_CODE', 
                                                                                   'INTERRUPTING_DEVICE']]

In [20]:
print("QC check 7", interruptingdevice_df.shape)
print("QC check 7", interruptingdevice_df.iloc[:, 0].nunique())

QC check 7 (4, 3)
QC check 7 4


In [21]:
interruptingdevice_df = interruptingdevice_df.rename(columns = {'ID': 'INTERRUPTING_DEVICE_ID'})

In [22]:
#servicecenter_df = servicecenter_df[servicecenter_df['SERVICECENTERSONLY'] == -1].iloc[:, [0, 1, 2, 3, 4, 7]].reset_index()

In [23]:
#Loading OUTAGE tables

In [24]:
customercalls_arc_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TCMS.TBLCUSTOMERCALLS_ARC_DPL.csv', sep = '|', encoding = "ISO-8859-1")
#EXCLUDED:  erthistory_df = pd.read_csv('gs://aes-datahub-0001-landing/OMS/DPL/TCMS.TBLERT_HISTORY.csv', sep =';')
#EXCLUDED:  erthistory_arc_df = pd.read_csv('gs://aes-datahub-0001-landing/OMS/DPL/TCMS.TBLERT_HISTORY_ARC.csv', sep = ';')
outages_arc_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLOUTAGES_ARC_DPL.csv', sep = '|', encoding = "ISO-8859-1")
outages_rolling_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLOUTAGES_ROLLING_DPL.csv', sep = '|', encoding = "ISO-8859-1")
restoresteps_arc_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLRESTORESTEPS_ARC_DPL.csv', sep = '|')
restoresteps_rolling_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLRESTORESTEPS_ROLLING_DPL.csv', sep = '|')
stormexclusions_m_df = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/DPL/TBLSTORMEXCLUSIONS_M_DPL.csv', sep = '|')

In [25]:
#Selecting OUTAGE-related variables

In [26]:
customercalls_arc_df = customercalls_arc_df.iloc[:, [0, 1, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 24, 25, 26, 35, 36, 38, 39, 40, 41, 42, 45, 53, 57]]

In [27]:
print("QC check 8", customercalls_arc_df.shape)
print("QC check 8", customercalls_arc_df.iloc[:, 0].nunique())

QC check 8 (2633307, 27)
QC check 8 2633306


In [28]:
customercalls_arc_df['DATETIMEREC'] = pd.to_datetime(customercalls_arc_df['DATETIMEREC'], infer_datetime_format = True, errors = 'coerce')
customercalls_arc_df['TIMEOFLASTCALL'] = pd.to_datetime(customercalls_arc_df['TIMEOFLASTCALL'], infer_datetime_format = True, errors = 'coerce')
customercalls_arc_df['DATETIMEREC'] = customercalls_arc_df[customercalls_arc_df['DATETIMEREC'] >= '2002-01-01 00:00:00']

In [29]:
print("QC check 9", customercalls_arc_df.shape)
print("QC check 9", customercalls_arc_df.iloc[:, 0].nunique())

QC check 9 (2633307, 27)
QC check 9 2633306


In [30]:
customercalls_arc_df = customercalls_arc_df.rename(columns = {'ID': 'CUSTOMERCALL_ID'})

In [31]:
outages_arc_df = outages_arc_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 54, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83]]
outages_arc_df['TIME_OF_FIRST_CALL'] = pd.to_datetime(outages_arc_df['TIME_OF_FIRST_CALL'], infer_datetime_format = True, errors = 'coerce')
outages_arc_df = outages_arc_df[outages_arc_df['TIME_OF_FIRST_CALL'] >= '2002-01-01 00:00:00']

In [32]:
print("QC check 10", outages_arc_df.shape)
print("QC check 10", outages_arc_df.iloc[:, 0].nunique())

QC check 10 (756623, 71)
QC check 10 756623


In [33]:
outages_rolling_df = outages_rolling_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 54, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83]]
outages_rolling_df['TIME_OF_FIRST_CALL'] = pd.to_datetime(outages_rolling_df['TIME_OF_FIRST_CALL'], infer_datetime_format = True, errors = 'coerce')

In [34]:
print("QC check 11", outages_rolling_df.shape)
print("QC check 11", outages_rolling_df.iloc[:, 0].nunique())

QC check 11 (40370, 71)
QC check 11 40370


In [35]:
outages_df = outages_arc_df.append(outages_rolling_df, ignore_index = True)

In [36]:
print("QC check 12", outages_df.shape)
print("QC check 12", outages_df.iloc[:, 0].nunique())

QC check 12 (796993, 71)
QC check 12 796993


In [37]:
outages_df['STORMDATE_JOIN'] = outages_df['TIME_OF_FIRST_CALL'].dt.date
outages_df['STORMDATE_JOIN'] = pd.to_datetime(outages_df['STORMDATE_JOIN'])

In [38]:
outages_df = outages_df.rename(columns = {'ID': 'OUTAGE_ID'})

In [39]:
restoresteps_arc_df = restoresteps_arc_df.iloc[:, [0, 1, 2 , 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 21, 24]]
restoresteps_rolling_df = restoresteps_rolling_df.iloc[:, [0, 1, 2 , 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 21, 24]]

In [40]:
print("QC check 13", restoresteps_arc_df.shape)
print("QC check 14", restoresteps_rolling_df.shape)
print("QC check 13", restoresteps_arc_df.iloc[:, 0].nunique())
print("QC check 14", restoresteps_rolling_df.iloc[:, 0].nunique())

QC check 13 (1149619, 17)
QC check 14 (283695, 17)
QC check 13 1149619
QC check 14 283695


In [41]:
restoresteps_arc_df['DATETIMEDISP'] = pd.to_datetime(restoresteps_arc_df['DATETIMEDISP'], infer_datetime_format = True, errors = 'coerce')
restoresteps_arc_df['DATETIMEARRV'] = pd.to_datetime(restoresteps_arc_df['DATETIMEARRV'], infer_datetime_format = True, errors = 'coerce')
restoresteps_arc_df['DATETIMERELEASED'] = pd.to_datetime(restoresteps_arc_df['DATETIMERELEASED'], infer_datetime_format = True, errors = 'coerce')

In [42]:
restoresteps_arc_df = restoresteps_arc_df[restoresteps_arc_df['DATETIMEDISP'] >= '2002-01-01 00:00:00']

In [43]:
print("QC check 15", restoresteps_arc_df.shape)
print("QC check 15", restoresteps_arc_df.iloc[:, 0].nunique())

QC check 15 (954244, 17)
QC check 15 954244


In [44]:
restoresteps_rolling_df['DATETIMEDISP'] = pd.to_datetime(restoresteps_rolling_df['DATETIMEDISP'], infer_datetime_format = True, errors = 'coerce')
restoresteps_rolling_df['DATETIMEARRV'] = pd.to_datetime(restoresteps_rolling_df['DATETIMEARRV'], infer_datetime_format = True, errors = 'coerce')

In [45]:
restore_df = restoresteps_arc_df.append(restoresteps_rolling_df, ignore_index = True)

In [46]:
stormexclusions_m_df = stormexclusions_m_df.iloc[:, 0:6]

In [47]:
print("QC check 16", stormexclusions_m_df.shape)
print("QC check 16", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 16 (164, 6)
QC check 16 164


In [48]:
stormexclusions_m_df['FROMDATE'] = pd.to_datetime(stormexclusions_m_df['FROMDATE'], infer_datetime_format = True, errors = 'coerce')
stormexclusions_m_df['TODATE'] = pd.to_datetime(stormexclusions_m_df['TODATE'], infer_datetime_format = True, errors = 'coerce')

In [49]:
stormexclusions_m_df = stormexclusions_m_df[stormexclusions_m_df['FROMDATE'] >= '2002-01-01'].reset_index(drop = True)

In [50]:
print("QC check 17", stormexclusions_m_df.shape)
print("QC check 17", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 17 (124, 6)
QC check 17 124


In [51]:
placeholder = []
for i in stormexclusions_m_df.iterrows():
    placeholder.append('-1')
    
placeholder = pd.DataFrame(placeholder)
placeholder.columns = ['STORM_FLAG']

stormexclusions_m_df = pd.concat([stormexclusions_m_df, placeholder], axis = 1)

In [52]:
print("QC check 18", stormexclusions_m_df.shape)
print("QC check 18", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 18 (124, 7)
QC check 18 124


In [53]:
stormexclusions_m_df['DATE1'] = stormexclusions_m_df['FROMDATE'].dt.date
stormexclusions_m_df['DATE2'] = stormexclusions_m_df['TODATE'].dt.date
stormexclusions_m_df['DATE_JOIN'] = ''

In [54]:
print("QC check 19", stormexclusions_m_df.shape)
print("QC check 19", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 19 (124, 10)
QC check 19 124


In [55]:
#Dropping duplicates because...
stormexclusions_m_df = stormexclusions_m_df.sort_values('FROMDATE').drop_duplicates('FROMDATE')

In [56]:
print("QC check 20", stormexclusions_m_df.shape)
print("QC check 20", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 20 (98, 10)
QC check 20 98


In [57]:
#Creates the mergeable tables called "storm_df"
columnnames = stormexclusions_m_df.columns.tolist()
storm_df = pd.DataFrame(columns = columnnames)

row_counter = 0
for i in stormexclusions_m_df.iterrows():
    date_list = pd.date_range(start = stormexclusions_m_df.iloc[row_counter, 7], 
                              end  = stormexclusions_m_df.iloc[row_counter, 8], 
                              freq = 'D').strftime('%Y-%m-%d').tolist()
    date_counter = 0
    for d in date_list:
        storm_df = storm_df.append(stormexclusions_m_df.iloc[row_counter, :], ignore_index = True)
        storm_df.iloc[-1, -1] = date_list[date_counter]
        date_counter = date_counter + 1
    row_counter = row_counter + 1

In [58]:
print("QC check 21", storm_df.shape)
print("QC check 21", storm_df.iloc[:, 0].nunique())

QC check 21 (175, 10)
QC check 21 98


In [59]:
storm_df['DATE_JOIN'] = pd.to_datetime(storm_df['DATE_JOIN'])

In [60]:
#Merging together the ADS

In [61]:
print("QC check 22", outages_df.shape)
print("QC check 22", outages_df.iloc[:, 0].nunique())

QC check 22 (796993, 72)
QC check 22 796993


In [62]:
#CAUSECODE to
    #OUTAGES
outages_df = outages_df.merge(causecode_df, left_on = 'CAUSE_ID', right_on = 'CAUSEID', how = 'left').drop('CAUSE_ID', axis = 1)

In [63]:
print("QC check 23", outages_df.shape)
print("QC check 23", outages_df.iloc[:, 0].nunique())

QC check 23 (796993, 77)
QC check 23 796993


In [64]:
#FAILEDDEVICE to
    #OUTAGES
outages_df = outages_df.merge(faileddevice_df, left_on = 'FAILDEV_ID', right_on = 'FAILED_DEVICE_ID', how = 'left').drop('FAILDEV_ID', axis = 1)

In [65]:
print("QC check 24", outages_df.shape)
print("QC check 24", outages_df.iloc[:, 0].nunique())

QC check 24 (796993, 79)
QC check 24 796993


In [66]:
#INTERRUPTINGDEVICE to
    #OUTAGES
outages_df = outages_df.merge(interruptingdevice_df, left_on = 'INTDEV_ID', right_on = 'INTERRUPTING_DEVICE_ID', how = 'left').drop('INTDEV_ID', axis = 1)

In [69]:
print("QC check 25", outages_df.shape)
print("QC check 25", outages_df.iloc[:, 0].nunique())

QC check 25 (796993, 81)
QC check 25 796993


In [70]:
#STORMEXCLUSIONS to
    #OUTAGES
storm_df = storm_df.drop('ID', axis = 1)
outages_df = outages_df.merge(storm_df, left_on = 'STORMDATE_JOIN', right_on = 'DATE_JOIN', how = 'left').drop('STORMDATE_JOIN', axis = 1)
outages_df = outages_df.drop(['FROMDATE', 'TODATE', 'DATE1', 'DATE2'], axis = 1)
outages_df = outages_df.rename(columns = {'DATE_JOIN': 'STORM_DATE'})

In [71]:
print("QC check 26", outages_df.shape)
print("QC check 26", outages_df.iloc[:, 0].nunique())

QC check 26 (796993, 85)
QC check 26 796993


In [72]:
print("QC check A1", restore_df.shape)
print("QC check A1", restore_df.iloc[:, 0].nunique())
print("QC check A1", outages_df.shape)
print("QC check A1", outages_df.iloc[:, 0].nunique())

QC check A1 (1237939, 17)
QC check A1 1237939
QC check A1 (796993, 85)
QC check A1 796993


In [73]:
#OUTAGES to
    #RESTORE
    
x = restore_df.OUTAGE_ID.unique().tolist()
y = outages_df.OUTAGE_ID.unique().tolist()
    
l = list(set(x).symmetric_difference(set(y)))

restore_df = restore_df[~restore_df['OUTAGE_ID'].isin(l)]
outages_df = outages_df[~outages_df['OUTAGE_ID'].isin(l)]
#Removing the 32 OUTAGE_ID's that do not overlap with the Outages table; and some of which account for an absurd amount of records

In [75]:
print("QC check A1", restore_df.shape)
print("QC check A1", restore_df.iloc[:, 0].nunique())
print("QC check A1", outages_df.shape)
print("QC check A1", outages_df.iloc[:, 0].nunique())

QC check A1 (1003166, 17)
QC check A1 1003166
QC check A1 (796983, 85)
QC check A1 796983


In [76]:
restoremerge_df = restore_df.groupby(['OUTAGE_ID'], as_index = False).agg({'CREWSIZE' : 'max',
                                                                        'DATETIMEDISP' : 'min',
                                                                        'DATETIMEARRV' : 'min'})

In [77]:
print("QC check 27", outages_df.shape)
print("QC check 27", outages_df.iloc[:, 0].nunique())
print("QC check 27", restoremerge_df.shape)
print("QC check 27", restoremerge_df.iloc[:, 0].nunique())

QC check 27 (796983, 85)
QC check 27 796983
QC check 27 (796983, 4)
QC check 27 796983


In [78]:
outages_df = outages_df.merge(restoremerge_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGE_ID', how = 'left')

In [79]:
print("QC check 27", outages_df.shape)
print("QC check 27", outages_df.iloc[:, 0].nunique())

QC check 27 (796983, 88)
QC check 27 796983


In [80]:
print("QC check 28", customercalls_arc_df.shape)
print("QC check 28", customercalls_arc_df.iloc[:, 0].nunique())

QC check 28 (2633307, 27)
QC check 28 2633306


In [81]:
#CALLTYPES to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(calltype_df, left_on = 'CALL_TYPE', right_on = 'ID', how = 'left').drop('ID', axis = 1)

In [82]:
print("QC check 29", customercalls_arc_df.shape)
print("QC check 29", customercalls_arc_df.iloc[:, 0].nunique())

QC check 29 (2633307, 28)
QC check 29 2633306


In [83]:
#DOWNLOAD_A to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(download_df, on = 'KY_PREM', how = 'left')

In [84]:
print("QC check 30", customercalls_arc_df.shape)
print("QC check 30", customercalls_arc_df.iloc[:, 0].nunique())

QC check 30 (2633307, 29)
QC check 30 2633306


In [85]:
#DEVICE to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(device_df, on = 'DEVICE', how = 'left')

In [86]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", customercalls_arc_df.iloc[:, 0].nunique())
print("QC check 31", outages_df.shape)
print("QC check 31", outages_df.iloc[:, 0].nunique())

QC check 31 (2633307, 95)
QC check 31 2633306
QC check 31 (796983, 88)
QC check 31 796983


In [87]:
#CUSTOMERCALLS to
    #OUTAGES

a = customercalls_arc_df.OUTAGEID.unique().tolist()
b = outages_df.OUTAGE_ID.unique().tolist()

list1 = list(set(a).symmetric_difference(set(b)))

customercalls_arc_df = customercalls_arc_df[~customercalls_arc_df['OUTAGEID'].isin(list1)]
outages_df = outages_df[~outages_df['OUTAGE_ID'].isin(list1)]

In [88]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", customercalls_arc_df.iloc[:, 0].nunique())
print("QC check 31", outages_df.shape)
print("QC check 31", outages_df.iloc[:, 0].nunique())

QC check 31 (2350213, 95)
QC check 31 2350212
QC check 31 (792819, 88)
QC check 31 792819


In [89]:
customercalls_arc_df = customercalls_arc_df.drop(['LATITUDE_x', 
                                                  'LONGITUDE_x', 
                                                  'CIRCUIT_y', 
                                                  'ACCOUNT_TYPE_x', 
                                                  'PHASE_A_x', 
                                                  'PHASE_B_x', 
                                                  'PHASE_C_x', 
                                                  'PHASE_T_x', 
                                                  'PHASE_A_y', 
                                                  'PHASE_B_y', 
                                                  'PHASE_C_y', 
                                                  'PHASE_T_y'], axis = 'columns')

In [90]:
print("QC check 32", customercalls_arc_df.shape)
print("QC check 32", customercalls_arc_df.iloc[:, 0].nunique())

QC check 32 (2350213, 83)
QC check 32 2350212


In [91]:
customercalls_arc_df = customercalls_arc_df.rename(columns = {'ACCOUNT_TYPE_y':'ACCOUNT_TYPE', 
                                                              'CIRCUIT_x':'CIRCUIT', 
                                                              'CALL_TYPE_x':'CALL_TYPE', 
                                                              'LATITUDE_y':'LATITUDE', 
                                                              'LONGITUDE_y':'LONGITUDE'})

In [92]:
customercalls_arc_df = pd.get_dummies(customercalls_arc_df, columns = ['DEVICETYPE', 
                                                                       'CALL_TYPE', 
                                                                       'ACCOUNT_TYPE', 
                                                                       'PHASE', 
                                                                       'PUBLIC_SAFETY',
                                                                       'SERVCTR', 
                                                                       'VOLTAGE'])

In [93]:
print("QC check 33", customercalls_arc_df.shape)
print("QC check 33", customercalls_arc_df.iloc[:, 0].nunique())

QC check 33 (2350213, 142)
QC check 33 2350212


In [94]:
customercalls_arc_df['OUTAGEID_JOIN'] = customercalls_arc_df['OUTAGEID']

In [95]:
print("QC check 34", customercalls_arc_df.shape)
print("QC check 34", customercalls_arc_df.iloc[:, 0].nunique())

QC check 34 (2350213, 143)
QC check 34 2350212


In [96]:
cc1_df = customercalls_arc_df.iloc[:, :-115].groupby(['OUTAGEID'],
                             as_index = False).agg({'CUSTOMERCALL_ID' : 'count',
                                                    ###'DATETIMEREC' : 'min',
                                                    ###CIRCUIT,
                                                    ###DEVICE,
                                                    'TIMEOFLASTCALL' : 'min',
                                                    'WIREDOWN_PTH' : 'count',
                                                    'WIREDOWN_PTP' : 'count',
                                                    'TREEONWIRE_PTH' : 'count',
                                                    'TREEONWIRE_PTP' : 'count',
                                                    'BROKENPOLE' : 'count',
                                                    'FIREEXPLOSION' : 'count',
                                                    'DAMAGEDTRANS' : 'count',
                                                    'KY_PREM' : 'median',
                                                    'CALLCOUNT' : 'median',
                                                    'MAJOR' : 'count',
                                                    ###SERV_CNTR,
                                                    ###UPLINE,
                                                    ###DEVICESIZE,
                                                    'TOPFUSE' : 'max',
                                                    'LENGTH' : 'max', 
                                                    'UNDERGD' : 'max',
                                                    'NORMALLYOPEN' : 'max',
                                                    ###CIRCUTTIE,
                                                    ###TIEUPLINE,
                                                    'DIRECTIONFWD' : 'max'})

### Triple pounds before the variale name means that I was unable to efficiently do the transformation - requesting assistance

In [97]:
cc2_df = customercalls_arc_df.iloc[:, -115:].groupby(['OUTAGEID_JOIN'],
                                            as_index = False).median()

In [98]:
#customercalls_df = 
customercalls_df = cc1_df.merge(cc2_df, left_on = 'OUTAGEID', right_on = 'OUTAGEID_JOIN', how = 'left').drop('OUTAGEID_JOIN', axis = 'columns')

In [99]:
print("QC check 35", customercalls_arc_df.shape)
print("QC check 35", customercalls_arc_df.iloc[:, 0].nunique())
print("QC check 35", outages_df.shape)
print("QC check 35", outages_df.iloc[:, 0].nunique())

QC check 35 (2350213, 143)
QC check 35 2350212
QC check 35 (792819, 88)
QC check 35 792819


In [ ]:
outages_df = outages_df.merge(customercalls_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGEID', how = 'left')

In [ ]:
print("QC check 35", outages_df.shape)

In [59]:
outages_df.to_csv('Outages_Dataset.csv', index = False)

In [59]:
outages_df.to_csv('Outages_Dataset.csv', index = False)

In [ ]:
print("QC check 35", outages_df.shape)

In [ ]:
outages_df = outages_df.merge(customercalls_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGEID', how = 'left')

In [ ]:
print("QC check 35", customercalls_arc_df.shape)
print("QC check 35", outages_df.shape)

In [ ]:
#customercalls_df = 
customercalls_df = cc1_df.merge(cc2_df, left_on = 'OUTAGEID', right_on = 'OUTAGEID_JOIN', how = 'left').drop('OUTAGEID_JOIN', axis = 'columns')

In [ ]:
cc2_df = customercalls_arc_df.iloc[:, -115:].groupby(['OUTAGEID_JOIN'],
                                            as_index = False).median()

In [ ]:
cc1_df = customercalls_arc_df.iloc[:, :-115].groupby(['OUTAGEID'],
                             as_index = False).agg({'CUSTOMERCALL_ID' : 'count',
                                                    ###'DATETIMEREC' : 'min',
                                                    ###CIRCUIT,
                                                    ###DEVICE,
                                                    'TIMEOFLASTCALL' : 'min',
                                                    'WIREDOWN_PTH' : 'count',
                                                    'WIREDOWN_PTP' : 'count',
                                                    'TREEONWIRE_PTH' : 'count',
                                                    'TREEONWIRE_PTP' : 'count',
                                                    'BROKENPOLE' : 'count',
                                                    'FIREEXPLOSION' : 'count',
                                                    'DAMAGEDTRANS' : 'count',
                                                    'KY_PREM' : 'median',
                                                    'CALLCOUNT' : 'median',
                                                    'MAJOR' : 'count',
                                                    ###SERV_CNTR,
                                                    ###UPLINE,
                                                    ###DEVICESIZE,
                                                    'TOPFUSE' : 'max',
                                                    'LENGTH' : 'max', 
                                                    'UNDERGD' : 'max',
                                                    'NORMALLYOPEN' : 'max',
                                                    ###CIRCUTTIE,
                                                    ###TIEUPLINE,
                                                    'DIRECTIONFWD' : 'max'})

### Triple pounds before the variale name means that I was unable to efficiently do the transformation - requesting assistance

In [ ]:
print("QC check 34", customercalls_arc_df.shape)

In [ ]:
customercalls_arc_df['OUTAGEID_JOIN'] = customercalls_arc_df['OUTAGEID']

In [90]:
print("QC check 33", customercalls_arc_df.shape)

SyntaxError: unexpected EOF while parsing (<ipython-input-90-191a6ee036f5>, line 1)

In [89]:
customercalls_arc_df = pd.get_dummies(customercalls_arc_df, columns = ['DEVICETYPE', 
                                                                       'CALL_TYPE', 
                                                                       'ACCOUNT_TYPE', 
                                                                       'PHASE', 
                                                                       'PUBLIC_SAFETY',
                                                                       'SERVCTR', 
                                                                       'VOLTAGE'])

In [88]:
customercalls_arc_df = customercalls_arc_df.rename(columns = {'ACCOUNT_TYPE_y':'ACCOUNT_TYPE', 
                                                              'CIRCUIT_x':'CIRCUIT', 
                                                              'CALL_TYPE_x':'CALL_TYPE', 
                                                              'LATITUDE_y':'LATITUDE', 
                                                              'LONGITUDE_y':'LONGITUDE'})

In [87]:
print("QC check 32", customercalls_arc_df.shape)

QC check 32 (2350213, 83)


In [86]:
customercalls_arc_df = customercalls_arc_df.drop(['LATITUDE_x', 
                                                  'LONGITUDE_x', 
                                                  'CIRCUIT_y', 
                                                  'ACCOUNT_TYPE_x', 
                                                  'PHASE_A_x', 
                                                  'PHASE_B_x', 
                                                  'PHASE_C_x', 
                                                  'PHASE_T_x', 
                                                  'PHASE_A_y', 
                                                  'PHASE_B_y', 
                                                  'PHASE_C_y', 
                                                  'PHASE_T_y'], axis = 'columns')

In [85]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", outages_df.shape)

QC check 31 (2350213, 95)
QC check 31 (792819, 88)


In [84]:
#CUSTOMERCALLS to
    #OUTAGES

a = customercalls_arc_df.OUTAGEID.unique().tolist()
b = outages_df.OUTAGE_ID.unique().tolist()

list1 = list(set(a).symmetric_difference(set(b)))

customercalls_arc_df = customercalls_arc_df[~customercalls_arc_df['OUTAGEID'].isin(list1)]
outages_df = outages_df[~outages_df['OUTAGE_ID'].isin(list1)]

In [83]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", outages_df.shape)

QC check 31 (2633307, 95)
QC check 31 (796983, 88)


In [82]:
#DEVICE to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(device_df, on = 'DEVICE', how = 'left')

In [81]:
print("QC check 30", customercalls_arc_df.shape)

QC check 30 (2633307, 29)


In [80]:
#DOWNLOAD_A to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(download_df, on = 'KY_PREM', how = 'left')

In [79]:
print("QC check 29", customercalls_arc_df.shape)

QC check 29 (2633307, 28)


In [78]:
#CALLTYPES to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(calltype_df, left_on = 'CALL_TYPE', right_on = 'ID', how = 'left').drop('ID', axis = 1)

In [77]:
print("QC check 28", customercalls_arc_df.shape)

QC check 28 (2633307, 27)


In [76]:
print("QC check 27", outages_df.shape)

QC check 27 (796983, 88)


In [75]:
outages_df = outages_df.merge(restoremerge_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGE_ID', how = 'left')

In [74]:
print("QC check 27", outages_df.shape)
print("QC check 27", restoremerge_df.shape)

QC check 27 (796983, 85)
QC check 27 (796983, 4)


In [73]:
restoremerge_df = restore_df.groupby(['OUTAGE_ID'], as_index = False).agg({'CREWSIZE' : 'max',
                                                                        'DATETIMEDISP' : 'min',
                                                                        'DATETIMEARRV' : 'min'})

In [72]:
print("QC check A1", restore_df.shape)
print("QC check A1", outages_df.shape)

QC check A1 (1003166, 17)
QC check A1 (796983, 85)


In [71]:
#OUTAGES to
    #RESTORE
    
x = restore_df.OUTAGE_ID.unique().tolist()
y = outages_df.OUTAGE_ID.unique().tolist()
    
l = list(set(x).symmetric_difference(set(y)))

restore_df = restore_df[~restore_df['OUTAGE_ID'].isin(l)]
outages_df = outages_df[~outages_df['OUTAGE_ID'].isin(l)]
#Removing the 32 OUTAGE_ID's that do not overlap with the Outages table; and some of which account for an absurd amount of records

In [70]:
print("QC check A1", restore_df.shape)
print("QC check A1", outages_df.shape)

QC check A1 (1237939, 17)
QC check A1 (796993, 85)


In [69]:
print("QC check 26", outages_df.shape)

QC check 26 (796993, 85)


In [68]:
#STORMEXCLUSIONS to
    #OUTAGES
storm_df = storm_df.drop('ID', axis = 1)
outages_df = outages_df.merge(storm_df, left_on = 'STORMDATE_JOIN', right_on = 'DATE_JOIN', how = 'left').drop('STORMDATE_JOIN', axis = 1)
outages_df = outages_df.drop(['FROMDATE', 'TODATE', 'DATE1', 'DATE2'], axis = 1)
outages_df = outages_df.rename(columns = {'DATE_JOIN': 'STORM_DATE'})

In [67]:
print("QC check 25", outages_df.shape)

QC check 25 (796993, 81)


In [133]:
#INTERRUPTINGDEVICE to
    #OUTAGES
outages_df = outages_df.merge(interruptingdevice_df, left_on = 'INTDEV_ID', right_on = 'INTERRUPTING_DEVICE_ID', how = 'left').drop('INTDEV_ID', axis = 1)

In [132]:
print("QC check 24", outages_df.shape)
print("QC check 24", outages_df.iloc[:, 0].nunique())

QC check 24 (796993, 79)
QC check 24 796993


In [131]:
#FAILEDDEVICE to
    #OUTAGES
outages_df = outages_df.merge(faileddevice_df, left_on = 'FAILDEV_ID', right_on = 'FAILED_DEVICE_ID', how = 'left').drop('FAILDEV_ID', axis = 1)

In [130]:
print("QC check 23", outages_df.shape)
print("QC check 23", outages_df.iloc[:, 0].nunique())

QC check 23 (796993, 77)
QC check 23 796993


In [129]:
#CAUSECODE to
    #OUTAGES
outages_df = outages_df.merge(causecode_df, left_on = 'CAUSE_ID', right_on = 'CAUSEID', how = 'left').drop('CAUSE_ID', axis = 1)

In [128]:
print("QC check 22", outages_df.shape)
print("QC check 22", outages_df.iloc[:, 0].nunique())

QC check 22 (796993, 72)
QC check 22 796993


In [127]:
#Merging together the ADS

In [126]:
storm_df['DATE_JOIN'] = pd.to_datetime(storm_df['DATE_JOIN'])

In [125]:
print("QC check 21", storm_df.shape)
print("QC check 21", storm_df.iloc[:, 0].nunique())

QC check 21 (175, 10)
QC check 21 98


In [124]:
#Creates the mergeable tables called "storm_df"
columnnames = stormexclusions_m_df.columns.tolist()
storm_df = pd.DataFrame(columns = columnnames)

row_counter = 0
for i in stormexclusions_m_df.iterrows():
    date_list = pd.date_range(start = stormexclusions_m_df.iloc[row_counter, 7], 
                              end  = stormexclusions_m_df.iloc[row_counter, 8], 
                              freq = 'D').strftime('%Y-%m-%d').tolist()
    date_counter = 0
    for d in date_list:
        storm_df = storm_df.append(stormexclusions_m_df.iloc[row_counter, :], ignore_index = True)
        storm_df.iloc[-1, -1] = date_list[date_counter]
        date_counter = date_counter + 1
    row_counter = row_counter + 1

In [123]:
print("QC check 20", stormexclusions_m_df.shape)
print("QC check 20", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 20 (98, 10)
QC check 20 98


In [122]:
stormexclusions_m_df = stormexclusions_m_df.sort_values('FROMDATE').drop_duplicates('FROMDATE')

In [121]:
print("QC check 19", stormexclusions_m_df.shape)
print("QC check 19", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 19 (124, 10)
QC check 19 124


In [120]:
stormexclusions_m_df['DATE1'] = stormexclusions_m_df['FROMDATE'].dt.date
stormexclusions_m_df['DATE2'] = stormexclusions_m_df['TODATE'].dt.date
stormexclusions_m_df['DATE_JOIN'] = ''

In [119]:
print("QC check 18", stormexclusions_m_df.shape)
print("QC check 18", stormexclusions_m_df.iloc[:, 0].nunique())

QC check 18 (124, 7)
QC check 18 124


In [118]:
placeholder = []
for i in stormexclusions_m_df.iterrows():
    placeholder.append('-1')
    
placeholder = pd.DataFrame(placeholder)
placeholder.columns = ['STORM_FLAG']

stormexclusions_m_df = pd.concat([stormexclusions_m_df, placeholder], axis = 1)

In [59]:
outages_df.to_csv('Outages_Dataset.csv', index = False)

In [ ]:
print("QC check 35", outages_df.shape)

In [ ]:
outages_df = outages_df.merge(customercalls_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGEID', how = 'left')

In [ ]:
print("QC check 35", customercalls_arc_df.shape)
print("QC check 35", outages_df.shape)

In [ ]:
#customercalls_df = 
customercalls_df = cc1_df.merge(cc2_df, left_on = 'OUTAGEID', right_on = 'OUTAGEID_JOIN', how = 'left').drop('OUTAGEID_JOIN', axis = 'columns')

In [ ]:
cc2_df = customercalls_arc_df.iloc[:, -115:].groupby(['OUTAGEID_JOIN'],
                                            as_index = False).median()

In [ ]:
cc1_df = customercalls_arc_df.iloc[:, :-115].groupby(['OUTAGEID'],
                             as_index = False).agg({'CUSTOMERCALL_ID' : 'count',
                                                    ###'DATETIMEREC' : 'min',
                                                    ###CIRCUIT,
                                                    ###DEVICE,
                                                    'TIMEOFLASTCALL' : 'min',
                                                    'WIREDOWN_PTH' : 'count',
                                                    'WIREDOWN_PTP' : 'count',
                                                    'TREEONWIRE_PTH' : 'count',
                                                    'TREEONWIRE_PTP' : 'count',
                                                    'BROKENPOLE' : 'count',
                                                    'FIREEXPLOSION' : 'count',
                                                    'DAMAGEDTRANS' : 'count',
                                                    'KY_PREM' : 'median',
                                                    'CALLCOUNT' : 'median',
                                                    'MAJOR' : 'count',
                                                    ###SERV_CNTR,
                                                    ###UPLINE,
                                                    ###DEVICESIZE,
                                                    'TOPFUSE' : 'max',
                                                    'LENGTH' : 'max', 
                                                    'UNDERGD' : 'max',
                                                    'NORMALLYOPEN' : 'max',
                                                    ###CIRCUTTIE,
                                                    ###TIEUPLINE,
                                                    'DIRECTIONFWD' : 'max'})

### Triple pounds before the variale name means that I was unable to efficiently do the transformation - requesting assistance

In [ ]:
print("QC check 34", customercalls_arc_df.shape)

In [ ]:
customercalls_arc_df['OUTAGEID_JOIN'] = customercalls_arc_df['OUTAGEID']

In [90]:
print("QC check 33", customercalls_arc_df.shape)

SyntaxError: unexpected EOF while parsing (<ipython-input-90-191a6ee036f5>, line 1)

In [89]:
customercalls_arc_df = pd.get_dummies(customercalls_arc_df, columns = ['DEVICETYPE', 
                                                                       'CALL_TYPE', 
                                                                       'ACCOUNT_TYPE', 
                                                                       'PHASE', 
                                                                       'PUBLIC_SAFETY',
                                                                       'SERVCTR', 
                                                                       'VOLTAGE'])

In [88]:
customercalls_arc_df = customercalls_arc_df.rename(columns = {'ACCOUNT_TYPE_y':'ACCOUNT_TYPE', 
                                                              'CIRCUIT_x':'CIRCUIT', 
                                                              'CALL_TYPE_x':'CALL_TYPE', 
                                                              'LATITUDE_y':'LATITUDE', 
                                                              'LONGITUDE_y':'LONGITUDE'})

In [87]:
print("QC check 32", customercalls_arc_df.shape)

QC check 32 (2350213, 83)


In [86]:
customercalls_arc_df = customercalls_arc_df.drop(['LATITUDE_x', 
                                                  'LONGITUDE_x', 
                                                  'CIRCUIT_y', 
                                                  'ACCOUNT_TYPE_x', 
                                                  'PHASE_A_x', 
                                                  'PHASE_B_x', 
                                                  'PHASE_C_x', 
                                                  'PHASE_T_x', 
                                                  'PHASE_A_y', 
                                                  'PHASE_B_y', 
                                                  'PHASE_C_y', 
                                                  'PHASE_T_y'], axis = 'columns')

In [85]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", outages_df.shape)

QC check 31 (2350213, 95)
QC check 31 (792819, 88)


In [84]:
#CUSTOMERCALLS to
    #OUTAGES

a = customercalls_arc_df.OUTAGEID.unique().tolist()
b = outages_df.OUTAGE_ID.unique().tolist()

list1 = list(set(a).symmetric_difference(set(b)))

customercalls_arc_df = customercalls_arc_df[~customercalls_arc_df['OUTAGEID'].isin(list1)]
outages_df = outages_df[~outages_df['OUTAGE_ID'].isin(list1)]

In [83]:
print("QC check 31", customercalls_arc_df.shape)
print("QC check 31", outages_df.shape)

QC check 31 (2633307, 95)
QC check 31 (796983, 88)


In [82]:
#DEVICE to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(device_df, on = 'DEVICE', how = 'left')

In [81]:
print("QC check 30", customercalls_arc_df.shape)

QC check 30 (2633307, 29)


In [80]:
#DOWNLOAD_A to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(download_df, on = 'KY_PREM', how = 'left')

In [79]:
print("QC check 29", customercalls_arc_df.shape)

QC check 29 (2633307, 28)


In [78]:
#CALLTYPES to
    #CUSTOMERCALLS

customercalls_arc_df = customercalls_arc_df.merge(calltype_df, left_on = 'CALL_TYPE', right_on = 'ID', how = 'left').drop('ID', axis = 1)

In [77]:
print("QC check 28", customercalls_arc_df.shape)

QC check 28 (2633307, 27)


In [76]:
print("QC check 27", outages_df.shape)

QC check 27 (796983, 88)


In [75]:
outages_df = outages_df.merge(restoremerge_df, left_on = 'OUTAGE_ID', right_on = 'OUTAGE_ID', how = 'left')